In [125]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
import numpy as np
import re
#删去不必要的列
def process(df):
    df.drop(['loan_id', 'user_id','scoring_low'], axis=1, inplace=True)
    return df

#将两个数据框丢掉各自独有的列
def regularize(df1,df2):
    df2.rename(columns={'is_default': 'isDefault'}, inplace=True)
    cols = set(df1.columns) - set(df2.columns)
    df1.drop(list(cols), axis=1, inplace=True)
    cols = set(df2.columns) - set(df1.columns)
    df2.drop(list(cols), axis=1, inplace=True)
    return df1,df2

#丢掉测试数据独有的
def test_process(train_data,test_data):
    cols = set(test_data.columns) - set(train_data.columns)
    test_data.drop(list(cols),axis=1,inplace=True)
    return test_data

#将两个数据框按行合并
def concat_df(df1, df2):  
    # 检查列是否相同  
    if set(df1.columns)==(set(df2.columns)):  
        # 给df1添加target1列  
        df1['target'] = 0
        # 给df2添加target0列  
        df2['target'] = 1
        # 合并数据框  
        result = pd.concat([df1, df2], ignore_index=True)  
        return result  
    else:  
        # 如果列不同，返回错误  
        print(df1.columns)
        print(df2.columns)
        raise ValueError("DataFrames have different columns.")

In [126]:
#数据预处理
def mean_fillna_process(df):
    #首先判断哪些列是数值类型，这些列进行平均值填充
    numeric_columns = df.select_dtypes(include=['number']).columns
    # 对每个数值类型的列进行平均值填充
    for column in numeric_columns:
        mean_value = df[column].mean()  # 计算平均值
        df[column].fillna(mean_value, inplace=True)  # 填充缺失值
    #对于其他类型的列，仍然采用众数填充
    non_numeric_columns = df.select_dtypes(exclude=['number']).columns
    for column in non_numeric_columns:
        mode_value = df[column].mode()[0]  # 计算众数，取第一个众数值
        df[column].fillna(mode_value, inplace=True)  # 填充缺失值
    return df

#worker_years预处理
def get_work_year(df):
    col = df["work_year"]
    new_work_year = []
    for ele in col:
        match = re.search(r'\S', ele)
        if match and match.group() == "<":
            new_work_year.append(0)
        else:
            match = re.search(r'\d+', ele)
            if match:
                new_work_year.append(int(match.group()))
            else:
                new_work_year.append(None)  # 如果没有匹配到数字，则添加 None
    df["new_work_year"] = new_work_year
    df.drop(["work_year"], axis=1, inplace=True)
    return df
#解析
def create_feature_process(df):
    #定义字典，键为12月份前三位，值从1到12
    month_dict = {
    "Jan": 1,"Feb": 2,"Mar": 3,"Apr": 4,
    "May": 5,"Jun": 6,"Jul": 7,"Aug": 8,
    "Sep": 9,"Oct": 10,"Nov": 11,"Dec": 12}
    col=df["earlies_credit_mon"]
    col_year=[]
    col_mon=[]
    for col_ele in col:
        list=col_ele.split("-")
        if list[0].isdigit():
            col_year.append((2000+int(list[0])))
            col_mon.append(month_dict[list[1]])
        else:
            col_year.append((1900+int(list[1])))
            col_mon.append(month_dict[list[0]])
    df["feature_year"]=col_year
    df["feature_month"]=col_mon
    df.drop(["earlies_credit_mon",],axis=1, inplace=True)
    return df

def create_feature_process2(df):
    month_dict = {
    "Jan": 1,"Feb": 2,"Mar": 3,"Apr": 4,
    "May": 5,"Jun": 6,"Jul": 7,"Aug": 8,
    "Sep": 9,"Oct": 10,"Nov": 11,"Dec": 12}
    col=df["earlies_credit_mon"]
    col_year=[]
    col_mon=[]
    for col_ele in col:
        list=col_ele.split("-")
        col_year.append(int(list[1]))
        col_mon.append(month_dict[list[0]])
    df["feature_year"]=col_year
    df["feature_month"]=col_mon
    df.drop(["earlies_credit_mon",],axis=1, inplace=True)
    return df

In [127]:
from sklearn.preprocessing import LabelEncoder
#字段类型的转变
transform_columns=["class","employer_type","industry"]
def transform_to_num(df,transform_columns):
    for col in transform_columns:
        #处理class类型
        class_map={'A': 0, 'B': 1, 'C': 2, 'D': 3,
        'E': 4, 'F': 5, 'G': 6}
        if col=="class":
            df[col] = df[col].map(class_map)
        else:
            df[col]=LabelEncoder().fit_transform(df[col])
    return df
    
            

In [128]:
public_data=process(TabularDataset('./train_public.csv'))
internet_data=process(TabularDataset('./train_internet.csv'))

Loaded data from: ./train_public.csv | Columns = 39 / 39 | Rows = 10000 -> 10000
Loaded data from: ./train_internet.csv | Columns = 42 / 42 | Rows = 750000 -> 750000


In [129]:
public_data

,total_loan,year_of_loan,interest,monthly_payment,class,employer_type,industry,work_year,house_exist,censor_status,...,policy_code,f0,f1,f2,f3,f4,early_return,early_return_amount,early_return_amount_3mon,isDefault
0,31818.18182,3,11.466,1174.91,C,政府机构,金融业,3 years,0,1,...,1,1.0,0.0,4.0,5.0,4.0,3,9927,0.0,0
1,28000.00000,5,16.841,670.69,C,政府机构,金融业,10+ years,0,2,...,1,7.0,0.0,4.0,45.0,22.0,0,0,0.0,0
2,17272.72727,3,8.900,603.32,A,政府机构,公共服务、社会组织,10+ years,1,0,...,1,6.0,0.0,6.0,28.0,19.0,0,0,0.0,0
3,20000.00000,3,4.788,602.30,A,世界五百强,文化和体育业,6 years,0,1,...,1,5.0,0.0,10.0,15.0,9.0,0,0,0.0,0
4,15272.72727,3,12.790,470.31,C,政府机构,信息传输、软件和信息技术服务业,< 1 year,2,1,...,1,10.0,0.0,6.0,15.0,4.0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,17727.27273,3,15.037,510.27,B,普通企业,建筑业,7 years,1,1,...,1,4.0,0.0,4.0,11.0,7.0,2,5287,0.0,0
9996,13636.36364,3,6.534,464.95,A,政府机构,农、林、牧、渔业,2 years,1,1,...,1,2.0,0.0,2.0,7.0,6.0,3,7182,0.0,0
9997,24818.18182,3,14.421,708.69,B,普通企业,信息传输、软件和信息技术服务业,10+ years,0,0,...,1,6.0,0.0,5.0,15.0,11.0,1,8540,2562.0,0
9998,20000.00000,3,18.450,727.58,D,政府机构,农、林、牧、渔业,10+ years,0,0,...,1,7.0,0.0,5.0,17.0,10.0,2,6161,616.1,0


In [130]:
internet_data

,total_loan,year_of_loan,interest,monthly_payment,class,sub_class,work_type,employer_type,industry,work_year,...,earlies_credit_mon,title,policy_code,f0,f1,f2,f3,f4,f5,is_default
0,12000.0,5,11.53,264.10,B,B5,职员,普通企业,采矿业,NaN,...,Mar-1984,0.0,1.0,1.0,0.0,8.0,17.0,8.0,1.0,1
1,8000.0,3,13.98,273.35,C,C3,其他,普通企业,国际组织,10+ years,...,Jan-1992,94.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0
2,20000.0,5,17.99,507.76,D,D2,工人,上市企业,信息传输、软件和信息技术服务业,10+ years,...,Oct-1996,0.0,1.0,6.0,0.0,10.0,8.0,3.0,0.0,0
3,10700.0,3,10.16,346.07,B,B1,职员,普通企业,电力、热力生产供应业,2 years,...,Jul-2000,41646.0,1.0,3.0,0.0,4.0,11.0,6.0,0.0,0
4,8000.0,3,8.24,251.58,B,B1,其他,政府机构,金融业,5 years,...,Mar-2000,4.0,1.0,3.0,0.0,8.0,6.0,4.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749995,12000.0,3,11.47,395.55,B,B5,职员,上市企业,文化和体育业,4 years,...,Jun-1995,0.0,1.0,6.0,0.0,8.0,22.0,12.0,5.0,0
749996,12000.0,3,6.03,365.23,A,A1,工人,政府机构,住宿和餐饮业,8 years,...,Sep-2001,4.0,1.0,2.0,0.0,7.0,4.0,4.0,0.0,0
749997,10000.0,3,15.41,348.67,D,D1,职员,政府机构,住宿和餐饮业,8 years,...,Oct-2008,8.0,1.0,6.0,0.0,10.0,11.0,3.0,0.0,0
749998,7200.0,3,9.44,230.44,B,B1,其他,政府机构,信息传输、软件和信息技术服务业,10+ years,...,Apr-2006,0.0,1.0,7.0,0.0,9.0,11.0,6.0,0.0,0


In [131]:
large_val_columns=["total_loan","monthly_payment","recircle_b","title","early_return_amount","early_return_amount_3mon","scoring_high"]
def log_process(df,columns):
    for col in columns:
        df[col]=np.log(df[col]+1)
    return df

In [132]:

#消除各自独有的列
regu_public_data,regu_internet_data=regularize(public_data,internet_data)
#分别进行处理
#缺失填充
mean_public_data=mean_fillna_process(regu_public_data)
mean_internet_data=mean_fillna_process(regu_internet_data)
#字符类型变换
trans_public_data=transform_to_num(mean_public_data,transform_columns)
trans_internet_data=transform_to_num(mean_internet_data,transform_columns)
#特征工程
log_public_data=log_process(trans_public_data,large_val_columns)
log_internet_data=log_process(trans_internet_data,large_val_columns)
#特征提取
wy_public_data=get_work_year(log_public_data)
wy_internet_data=get_work_year(log_internet_data)
pub_data=create_feature_process(wy_public_data)
inter_data=create_feature_process2(wy_internet_data)



C:\Users\woczgl\AppData\Local\Temp\ipykernel_14752\1738779828.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(mean_value, inplace=True)  # 填充缺失值
C:\Users\woczgl\AppData\Local\Temp\ipykernel_14752\1738779828.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.



In [133]:
pub_data

,total_loan,year_of_loan,interest,monthly_payment,class,employer_type,industry,house_exist,censor_status,issue_date,...,f2,f3,f4,early_return,early_return_amount,early_return_amount_3mon,isDefault,new_work_year,feature_year,feature_month
0,10.367825,3,11.466,7.069798,2,3,13,0,1,2016/10/1,...,4.0,5.0,4.0,3,9.203114,0.000000,0,3,2001,12
1,10.239996,5,16.841,6.509797,2,3,13,0,2,2013/6/1,...,4.0,45.0,22.0,0,0.000000,0.000000,0,10,1990,4
2,9.756942,3,8.900,6.404104,0,3,3,1,0,2014/1/1,...,6.0,28.0,19.0,0,0.000000,0.000000,0,10,1991,10
3,9.903538,3,4.788,6.402415,0,1,10,0,1,2015/7/1,...,10.0,15.0,9.0,0,0.000000,0.000000,0,6,2001,6
4,9.633889,3,12.790,6.155516,2,3,2,2,1,2016/7/1,...,6.0,15.0,4.0,0,0.000000,0.000000,0,0,2002,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9.782916,3,15.037,6.236898,1,4,7,1,1,2013/11/1,...,4.0,11.0,7.0,2,8.573195,0.000000,0,7,2006,2
9996,9.520569,3,6.534,6.144078,0,3,4,1,1,2015/12/1,...,2.0,7.0,6.0,3,8.879472,0.000000,0,2,1997,5
9997,10.119372,3,14.421,6.564828,1,4,2,0,0,2012/12/1,...,5.0,15.0,11.0,1,9.052633,7.848934,0,10,1987,2
9998,9.903538,3,18.450,6.591097,3,3,4,0,0,2018/3/1,...,5.0,17.0,10.0,2,8.726157,6.425031,0,10,1992,10


In [134]:
inter_data

,total_loan,year_of_loan,interest,monthly_payment,class,employer_type,industry,house_exist,censor_status,issue_date,...,policy_code,f0,f1,f2,f3,f4,isDefault,new_work_year,feature_year,feature_month
0,9.392745,5,11.53,5.580107,1,4,12,0,2,2015-06-01,...,1.0,1.000000,0.000000,8.00000,17.000000,8.000000,1,10,1984,3
1,8.987322,3,13.98,5.614405,2,4,6,0,2,2010-10-01,...,1.0,5.592246,0.000805,8.57743,14.624197,8.110185,0,10,1992,1
2,9.903538,5,17.99,6.231976,3,0,2,0,1,2016-08-01,...,1.0,6.000000,0.000000,10.00000,8.000000,3.000000,0,10,1996,10
3,9.278092,3,10.16,5.849526,1,4,11,2,2,2013-05-01,...,1.0,3.000000,0.000000,4.00000,11.000000,6.000000,0,2,2000,7
4,8.987322,3,8.24,5.531728,1,3,13,1,0,2017-04-01,...,1.0,3.000000,0.000000,8.00000,6.000000,4.000000,0,5,2000,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749995,9.392745,3,11.47,5.982802,1,0,10,0,1,2016-02-01,...,1.0,6.000000,0.000000,8.00000,22.000000,12.000000,0,4,1995,6
749996,9.392745,3,6.03,5.903262,0,3,1,1,2,2014-03-01,...,1.0,2.000000,0.000000,7.00000,4.000000,4.000000,0,8,2001,9
749997,9.210440,3,15.41,5.856990,3,3,1,1,2,2015-12-01,...,1.0,6.000000,0.000000,10.00000,11.000000,3.000000,0,8,2008,10
749998,8.881975,3,9.44,5.444321,1,3,2,2,2,2017-12-01,...,1.0,7.000000,0.000000,9.00000,11.000000,6.000000,0,10,2006,4


In [135]:
print(pub_data.dtypes)

total_loan                  float64
year_of_loan                  int64
interest                    float64
monthly_payment             float64
class                         int64
employer_type                 int32
industry                      int32
house_exist                   int64
censor_status                 int64
issue_date                   object
use                           int64
post_code                     int64
region                        int64
debt_loan_ratio             float64
del_in_18month                int64
scoring_high                float64
pub_dero_bankrup            float64
recircle_b                  float64
recircle_u                  float64
initial_list_status           int64
title                       float64
policy_code                   int64
f0                          float64
f1                          float64
f2                          float64
f3                          float64
f4                          float64
early_return                

In [136]:
print(inter_data.dtypes)

total_loan                  float64
year_of_loan                  int64
interest                    float64
monthly_payment             float64
class                         int64
employer_type                 int32
industry                      int32
house_exist                   int64
censor_status                 int64
issue_date                   object
use                           int64
post_code                   float64
region                        int64
debt_loan_ratio             float64
del_in_18month              float64
scoring_high                float64
pub_dero_bankrup            float64
early_return                  int64
early_return_amount         float64
early_return_amount_3mon    float64
recircle_b                  float64
recircle_u                  float64
initial_list_status           int64
title                       float64
policy_code                 float64
f0                          float64
f1                          float64
f2                          

In [137]:
#统一类型
def unify(df1, df2):
    # 将所有数值类型统一到df1的数值类型
    numeric_cols_df1 = df1.select_dtypes(include='number').columns
    # 将df2中数值类型列转换为df1的数值类型
    numeric_cols_df2 = df2.select_dtypes(include='number').columns
    for col in numeric_cols_df2:
        if col in numeric_cols_df1:
            df2[col] = df2[col].astype(df1[col].dtype)
    # 将internet数据的issue_date列转化为斜杠形式
    df2['issue_date'] = pd.to_datetime(df2['issue_date']).dt.strftime('%Y/%m/%d')

    return df1, df2


In [138]:
pub_data,inter_data=unify(pub_data,inter_data)

In [139]:
#合并数据
merge_data=concat_df(public_data,internet_data)

In [140]:
merge_data

,total_loan,year_of_loan,interest,monthly_payment,class,employer_type,industry,house_exist,censor_status,issue_date,...,f3,f4,early_return,early_return_amount,early_return_amount_3mon,isDefault,new_work_year,feature_year,feature_month,target
0,10.367825,3,11.466,7.069798,2,3,13,0,1,2016/10/1,...,5.0,4.0,3,9.203114,0.00000,0,3,2001,12,0
1,10.239996,5,16.841,6.509797,2,3,13,0,2,2013/6/1,...,45.0,22.0,0,0.000000,0.00000,0,10,1990,4,0
2,9.756942,3,8.900,6.404104,0,3,3,1,0,2014/1/1,...,28.0,19.0,0,0.000000,0.00000,0,10,1991,10,0
3,9.903538,3,4.788,6.402415,0,1,10,0,1,2015/7/1,...,15.0,9.0,0,0.000000,0.00000,0,6,2001,6,0
4,9.633889,3,12.790,6.155516,2,3,2,2,1,2016/7/1,...,15.0,4.0,0,0.000000,0.00000,0,0,2002,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759995,9.392745,3,11.470,5.982802,1,0,10,0,1,2016/02/01,...,22.0,12.0,0,0.000000,0.00000,0,4,1995,6,1
759996,9.392745,3,6.030,5.903262,0,3,1,1,2,2014/03/01,...,4.0,4.0,6,6.698268,5.86902,0,8,2001,9,1
759997,9.210440,3,15.410,5.856990,3,3,1,1,2,2015/12/01,...,11.0,3.0,0,0.000000,0.00000,0,8,2008,10,1
759998,8.881975,3,9.440,5.444321,1,3,2,2,2,2017/12/01,...,11.0,6.0,0,0.000000,0.00000,0,10,2006,4,1


In [141]:
#二分类问题
hyperparameters = {
    'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}, {}, 'GBMLarge'],
    'CAT': {'iterations': 50, 'learning_rate': 0.1, 'depth': 6},
    'XGB': {'n_estimators': 50, 'learning_rate': 0.1, 'max_depth': 6},
}


predictor_p_i = TabularPredictor(label='target',eval_metric="roc_auc").fit(merge_data.head(12000), presets='medium_quality',hyperparameters=hyperparameters)

No path specified. Models will be saved in: "AutogluonModels\ag-20240507_190505"
Presets specified: ['medium_quality']
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20240507_190505"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.9.12
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          12
Memory Avail:       2.38 GB / 15.86 GB (15.0%)
Disk Space Avail:   18.72 GB / 200.00 GB (9.4%)
Train Data Rows:    12000
Train Data Columns: 34
Label Column:       target
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Problem Type:       binary
Preprocessing data ...
Selected cla

In [142]:
predictor_p_i.leaderboard()


,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.999810,roc_auc,0.007005,1.341226,0.000000,0.058004,2,True,6
1,LightGBM,0.999810,roc_auc,0.007005,1.283221,0.007005,1.283221,1,True,2
2,LightGBMLarge,0.999525,roc_auc,0.005999,2.001281,0.005999,2.001281,1,True,5
3,XGBoost,0.995320,roc_auc,0.005006,0.165813,0.005006,0.165813,1,True,4
4,CatBoost,0.992055,roc_auc,0.002174,0.327661,0.002174,0.327661,1,True,3
5,LightGBMXT,0.979180,roc_auc,0.010752,1.713536,0.010752,1.713536,1,True,1


In [143]:
predictor_p_i.feature_importance(merge_data.head(12000))

These features in provided data are not utilized by the predictor and will be ignored: ['policy_code']
Computing feature importance via permutation shuffling for 33 features using 5000 rows with 5 shuffle sets...
	7.2s	= Expected runtime (1.44s per shuffle set)
	4.04s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
scoring_high,2.360167e-01,1.214824e-02,8.393266e-07,5,2.610300e-01,2.110033e-01
early_return_amount,6.760091e-02,2.961158e-03,4.406634e-07,5,7.369797e-02,6.150384e-02
early_return,4.779187e-03,5.649215e-04,2.299693e-05,5,5.942368e-03,3.616006e-03
recircle_u,5.880243e-04,1.569876e-04,5.557452e-04,5,9.112638e-04,2.647847e-04
f1,2.693958e-04,3.823513e-05,4.741252e-05,5,3.481224e-04,1.906691e-04
interest,1.784340e-04,7.778051e-05,3.419973e-03,5,3.385851e-04,1.828280e-05
feature_year,7.185193e-05,8.055889e-05,5.843151e-02,5,2.377238e-04,-9.401995e-05
f0,5.459145e-06,4.093566e-06,2.032915e-02,5,1.388785e-05,-2.969564e-06
isDefault,3.903168e-06,3.435636e-06,3.197826e-02,5,1.097719e-05,-3.170854e-06
total_loan,3.181244e-06,3.622075e-06,6.050456e-02,5,1.063915e-05,-4.276659e-06


In [144]:
test_inter_data=inter_data.drop(columns="target").tail(740000)

In [145]:
pred_y = predictor_p_i.predict_proba(test_inter_data)

In [146]:
test_inter_data["pred_y"]=pred_y[1]
add_to_public=test_inter_data[test_inter_data["pred_y"]<0.08]
add_to_public

,total_loan,year_of_loan,interest,monthly_payment,class,employer_type,industry,house_exist,censor_status,issue_date,...,f0,f1,f2,f3,f4,isDefault,new_work_year,feature_year,feature_month,pred_y
10258,9.386476,3,15.61,6.035386,3,1,11,0,1,2015/06/01,...,4.0,0.0,3.0,10.0,8.0,1,9,1995,11,0.000915
10267,7.937732,3,14.85,4.583538,2,3,7,0,2,2015/12/01,...,7.0,0.0,2.0,16.0,8.0,0,10,2001,7,0.010539
10497,8.657129,3,21.49,5.389483,3,4,3,1,2,2016/08/01,...,5.0,0.0,1.0,11.0,5.0,1,10,2003,10,0.073225
10547,8.006701,3,12.59,4.620059,2,3,13,1,2,2015/12/01,...,3.0,0.0,4.0,18.0,12.0,0,10,1998,9,0.060726
11039,9.798183,3,10.16,6.368479,1,4,13,1,0,2013/07/01,...,3.0,0.0,6.0,11.0,10.0,0,10,1994,12,0.011659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749795,7.313887,3,10.99,3.914221,1,3,11,2,0,2016/10/01,...,3.0,0.0,6.0,30.0,10.0,0,10,2004,11,0.000718
749839,9.729194,5,16.59,6.027869,3,3,8,1,0,2016/01/01,...,5.0,0.0,4.0,10.0,6.0,1,8,2001,3,0.004823
749920,9.210440,3,14.08,5.838226,2,4,2,1,1,2017/11/01,...,5.0,0.0,3.0,7.0,5.0,0,1,2008,1,0.011470
749924,9.862718,3,7.90,6.399892,0,0,1,2,0,2014/02/01,...,5.0,0.0,5.0,29.0,19.0,0,5,1999,9,0.028884


In [147]:

add_to_public=add_to_public.drop(columns="pred_y")
pub_data=pub_data.drop(columns="target")
train_data=pd.concat([pub_data, add_to_public], ignore_index=True)
train_data


,total_loan,year_of_loan,interest,monthly_payment,class,employer_type,industry,house_exist,censor_status,issue_date,...,f2,f3,f4,early_return,early_return_amount,early_return_amount_3mon,isDefault,new_work_year,feature_year,feature_month
0,10.367825,3,11.466,7.069798,2,3,13,0,1,2016/10/1,...,4.0,5.0,4.0,3,9.203114,0.000000,0,3,2001,12
1,10.239996,5,16.841,6.509797,2,3,13,0,2,2013/6/1,...,4.0,45.0,22.0,0,0.000000,0.000000,0,10,1990,4
2,9.756942,3,8.900,6.404104,0,3,3,1,0,2014/1/1,...,6.0,28.0,19.0,0,0.000000,0.000000,0,10,1991,10
3,9.903538,3,4.788,6.402415,0,1,10,0,1,2015/7/1,...,10.0,15.0,9.0,0,0.000000,0.000000,0,6,2001,6
4,9.633889,3,12.790,6.155516,2,3,2,2,1,2016/7/1,...,6.0,15.0,4.0,0,0.000000,0.000000,0,0,2002,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17668,7.313887,3,10.990,3.914221,1,3,11,2,0,2016/10/01,...,6.0,30.0,10.0,4,6.364751,5.788144,0,10,2004,11
17669,9.729194,5,16.590,6.027869,3,3,8,1,0,2016/01/01,...,4.0,10.0,6.0,0,0.000000,0.000000,1,8,2001,3
17670,9.210440,3,14.080,5.838226,2,4,2,1,1,2017/11/01,...,3.0,7.0,5.0,5,6.545350,5.943009,0,1,2008,1
17671,9.862718,3,7.900,6.399892,0,0,1,2,0,2014/02/01,...,5.0,29.0,19.0,0,0.000000,0.000000,0,5,1999,9


In [150]:
label = 'isDefault'
hyperparameters = {
    'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}, {}, 'GBMLarge'],
    'CAT': {'iterations': 100, 'learning_rate': 0.1, 'depth': 10},
    'XGB': {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 10}
}
predictor = TabularPredictor(label=label,eval_metric="roc_auc").fit(train_data, presets='medium_quality',hyperparameters=hyperparameters,auto_stack=True)


No path specified. Models will be saved in: "AutogluonModels\ag-20240507_190807"
Presets specified: ['medium_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20240507_190807"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.9.12
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          12
Memory Avail:       2.52 GB / 15.86 GB (15.9%)
Disk Space Avail:   18.67 GB / 200.00 GB (9.3%)
Train Data Rows:    17673
Train Data Columns: 33
Label Column:       isDefault
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary

In [151]:
predictor.leaderboard()

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.863073,roc_auc,0.844837,38.193493,0.002879,0.256001,2,True,6
1,LightGBM_BAG_L1,0.860160,roc_auc,0.131539,3.148246,0.131539,3.148246,1,True,2
2,LightGBMLarge_BAG_L1,0.859336,roc_auc,0.213552,5.118039,0.213552,5.118039,1,True,5
3,LightGBMXT_BAG_L1,0.857308,roc_auc,0.335640,4.403401,0.335640,4.403401,1,True,1
4,XGBoost_BAG_L1,0.855515,roc_auc,0.133060,4.367247,0.133060,4.367247,1,True,4
5,CatBoost_BAG_L1,0.854322,roc_auc,0.028167,20.900558,0.028167,20.900558,1,True,3


In [152]:
test_data=TabularDataset('./test_public.csv')
mean_test_data=mean_fillna_process(test_data)
trans_test_data=transform_to_num(mean_test_data,transform_columns)
log_test_data=log_process(trans_test_data,large_val_columns)
wy_test_data=get_work_year(log_test_data)
te_data=create_feature_process(wy_test_data)
te_data

Loaded data from: ./test_public.csv | Columns = 38 / 38 | Rows = 5000 -> 5000
C:\Users\woczgl\AppData\Local\Temp\ipykernel_14752\1738779828.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(mean_value, inplace=True)  # 填充缺失值
C:\Users\woczgl\AppData\Local\Temp\ipykernel_14752\1738779828.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the 

,loan_id,user_id,total_loan,year_of_loan,interest,monthly_payment,class,employer_type,industry,house_exist,...,f1,f2,f3,f4,early_return,early_return_amount,early_return_amount_3mon,new_work_year,feature_year,feature_month
0,1000575,200575,7.969672,3,10.791,4.488749,1,2,1,0,...,0.00000,15.000000,5.000000,4.00000,3,6.651572,4.501944,5,2003,3
1,1028125,228125,8.892024,3,9.990,5.557214,1,4,9,1,...,0.00000,8.000000,29.000000,14.00000,1,7.546974,5.391527,10,1999,12
2,1010694,210694,10.177189,3,15.763,6.639915,2,4,1,0,...,0.00000,4.000000,10.000000,6.00000,1,8.643121,7.108433,10,1999,4
3,1026712,226712,10.029764,5,19.305,6.263970,3,4,12,0,...,0.00000,12.000000,10.000000,8.00000,2,8.476580,6.095998,10,1900,7
4,1002895,202895,9.585103,3,7.139,6.197096,0,1,13,0,...,0.00000,7.000000,14.000000,9.00000,0,8.165364,6.477138,1,2007,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,1008856,208856,9.154357,5,12.015,5.217487,2,3,2,1,...,0.00088,8.589538,14.703649,8.13204,0,0.000000,0.000000,0,2005,7
4996,1016651,216651,8.612685,3,7.970,5.154909,0,3,8,1,...,0.00000,2.000000,5.000000,3.00000,3,7.355641,0.000000,5,2012,4
4997,1024140,224140,10.327004,3,8.900,6.791323,0,0,8,0,...,0.00000,17.000000,20.000000,14.00000,2,8.604654,7.321117,10,1986,10
4998,1014316,214316,8.316767,3,6.030,5.031614,0,3,10,0,...,0.00000,6.000000,10.000000,10.00000,3,5.411646,3.741691,10,1999,3


In [153]:
predict_result= predictor.predict_proba(te_data)
df2 = predict_result
df = pd.concat([te_data,df2],axis=1)
df = df[['loan_id',1]]
df = df.rename(columns={'loan_id':'id',1:'isDefault'})
df.to_csv('submission.csv')

Loaded data from: ./train_public.csv | Columns = 39 / 39 | Rows = 10000 -> 10000
Loaded data from: ./train_internet.csv | Columns = 42 / 42 | Rows = 750000 -> 750000
